## 건강 검진 데이터를 바탕으로 lDL 콜레스트롤 수치를 예측해보는 선형 회귀 모델

### 가설 1. 치아 관련 문항은 콜레스트롤과 상관이 없을 것이다.

In [22]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv("국민건강보험공단_건강검진정보_20221231.csv", encoding='cp949')

#pandas display 옵션조정을 통해 View 범위 확장하기
pd.set_option('display.max_columns', None)

#데이터 확인하기
df.head()

,기준년도,가입자일련번호,시도코드,성별,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,이완기혈압,식전혈당(공복혈당),총콜레스테롤,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,구강검진수검여부,치아우식증유무,치석
0,2022,702763,41,2,14,160,75,89.0,1.0,1.0,1.0,1.0,127.0,73.0,84.0,NaN,NaN,NaN,NaN,14.8,1.0,0.8,25.0,20.0,15.0,1.0,0.0,0,NaN,NaN
1,2022,765435,41,2,9,150,45,68.0,1.0,1.0,1.0,1.0,127.0,75.0,85.0,NaN,NaN,NaN,NaN,13.1,1.0,0.8,19.0,10.0,7.0,1.0,1.0,0,NaN,NaN
2,2022,343621,41,2,13,150,60,78.8,1.0,1.0,1.0,1.0,135.0,81.0,105.0,NaN,NaN,NaN,NaN,13.0,1.0,0.8,17.0,15.0,15.0,1.0,1.0,0,NaN,NaN
3,2022,3393534,41,2,6,160,50,69.0,1.0,1.0,1.0,1.0,109.0,70.0,81.0,NaN,NaN,NaN,NaN,13.4,1.0,0.8,25.0,16.0,13.0,1.0,1.0,0,NaN,NaN
4,2022,1716951,41,2,7,160,55,75.0,1.0,1.0,1.0,1.0,110.0,67.0,79.0,NaN,NaN,NaN,NaN,14.7,1.0,0.8,22.0,22.0,15.0,1.0,1.0,0,NaN,NaN


In [23]:
#가설을 참고하여 데이터 일부 삭제하기
# 시력, 청력, 치아 관련 칼럼은 관계없다는 가정으로 열 제거하기
df.drop(['치아우식증유무', '치석', '시력(좌)', '시력(우)', '청력(좌)', '청력(우)', '구강검진수검여부'], axis=1, inplace=True)

#기준년도 칼럼 확인하기(모두 '2022' 동일 값이므로 '기준년도' 칼럼 삭제)
print("기준년도 칼럼 확인")
df.기준년도.value_counts()

기준년도 칼럼 확인


2022    1000000
Name: 기준년도, dtype: int64

In [24]:
#가입자 일련번호 칼럼 확인하기(모두 unique한 값으로 확인되어 '가입자 일련번호' 칼럼 삭제)
print("가입자 일련번호 칼럼 확인")
df["가입자일련번호"].value_counts()

가입자 일련번호 칼럼 확인


702763     1
3341688    1
4871195    1
4610838    1
3997948    1
          ..
3308368    1
371852     1
3479774    1
3645306    1
4901217    1
Name: 가입자일련번호, Length: 1000000, dtype: int64

In [25]:
#성별코드 칼럼 확인하기
print("성별 칼럼 확인")
df.성별.value_counts()

성별 칼럼 확인


1    515301
2    484699
Name: 성별, dtype: int64

In [26]:
#불필요한 데이터 삭제하기
df.drop(['기준년도', '가입자일련번호', '성별', '시도코드'], axis=1, inplace=True)

#별도의 test 데이터 추출하기
test = df[df['LDL콜레스테롤'].isnull()]

#NaN 데이터 행 단위로 삭제하기
train = df.dropna(axis=0)

#학습 데이터 확인하기
print("학습 데이터 확인")
train.head(1)

학습 데이터 확인


,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,수축기혈압,이완기혈압,식전혈당(공복혈당),총콜레스테롤,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부
654336,13,165,60,80.0,118.0,68.0,103.0,243.0,120.0,85.0,134.0,14.0,1.0,1.0,22.0,17.0,34.0,2.0,1.0


In [27]:
#정답 데이터 생성하기
y = train['LDL콜레스테롤']

#학습 데이터 생성하기
x = train.drop('LDL콜레스테롤', axis=1)

#validation set 추출을 위한 train_test_split 라이브러리 불러오기
from sklearn.model_selection import train_test_split

#scikit learn 예시 코드 비율대로 불러오기
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

#학습/검증 데이터 확인하기
print("학습/검증 데이터 확인")
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

학습/검증 데이터 확인
(225877, 18) (225877,) (111253, 18) (111253,)


In [29]:
# 선형 회귀 학습 및 추론하기

In [31]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

#학습하기
reg.fit(x_train, y_train)
#기울기와 절편 확인하기
print(f'''기울기 및 절편 확인
기울기 확인 coef = {reg.coef_}
절편 확인 intercept = {reg.intercept_}''')


기울기 및 절편 확인
기울기 확인 coef = [-0.00211547  0.00838286  0.00403974  0.02586561 -0.00362227 -0.00478919
 -0.00181933  0.98893442 -0.18998983 -0.90963871  0.04906444  0.04162509
  0.14300268 -0.00654572  0.00520954 -0.00195242  0.08355305 -0.22383297]
절편 확인 intercept = -7.701480658594477


In [33]:
#각각 데이터에 대해 가중치 확인하기
print("전체에 대해서 가중치 확인")
for index, columns in enumerate(x_train.columns):
    print(f"{columns} = {reg.coef_[index]}")

전체에 대해서 가중치 확인
연령대코드(5세단위) = -0.0021154736576941484
신장(5cm단위) = 0.008382856140952737
체중(5kg단위) = 0.004039744108514473
허리둘레 = 0.02586561239950298
수축기혈압 = -0.003622269066452875
이완기혈압 = -0.004789193384671206
식전혈당(공복혈당) = -0.0018193256235434347
총콜레스테롤 = 0.9889344164631596
트리글리세라이드 = -0.18998982863839814
HDL콜레스테롤 = -0.9096387125982947
혈색소 = 0.04906444369407475
요단백 = 0.04162509002268398
혈청크레아티닌 = 0.14300268237983213
혈청지오티(AST) = -0.006545718172645947
혈청지피티(ALT) = 0.005209544960069898
감마지티피 = -0.0019524161439548073
흡연상태 = 0.08355304678253961
음주여부 = -0.22383296735907007


In [34]:
#예측을 통한 최종 검증하기

In [35]:
#예측하기
y_pred = reg.predict(x_test)

#결과 검증을 위해 MSE 라이브러리 불러오기
#최종적으로는 RMSE 사용하기
from sklearn.metrics import mean_squared_error

#MSE 라이브러리에서 RMSE는 squared 옵션을 False로 설정
rmse = mean_squared_error(y_test, y_pred, squared=False)

#주요 Feature 삭제 전 rmse 확인하기
print(f'''주요 Feature 삭제 전 rmse = {round(rmse,3)}''')

주요 Feature 삭제 전 rmse = 4.293


In [36]:
#가중치의 의미 확인하기

In [37]:
# 주요 Feature 삭제를 위해 칼럼명 재확인하기
train.columns

Index(['연령대코드(5세단위)', '신장(5cm단위)', '체중(5kg단위)', '허리둘레', '수축기혈압', '이완기혈압',
       '식전혈당(공복혈당)', '총콜레스테롤', '트리글리세라이드', 'HDL콜레스테롤', 'LDL콜레스테롤', '혈색소',
       '요단백', '혈청크레아티닌', '혈청지오티(AST)', '혈청지피티(ALT)', '감마지티피', '흡연상태', '음주여부'],
      dtype='object')

In [39]:
#주요 Feature 삭제하기
x = x.drop(['총콜레스테롤', '트리글리세라이드', 'HDL콜레스테롤'], axis=1)

#scikit learn에서 코드 비율대로 불러오기
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [41]:
#선형 회귀 재생성 및 학습하기
lr = LinearRegression()
lr.fit(x_train, y_train)

#각각 데이터에 대해 가중치 확인하기
print(f'''coef
{lr.coef_}
intercept
{lr.intercept_}''')

coef
[-1.50293683 -0.39035452  0.15513633 -0.20859166 -0.20332664  0.40434892
 -0.13615129  3.43928937 -0.68370857 -2.45956598 -0.08639088  0.06739086
 -0.01048069 -2.77843294  0.15151187]
intercept
170.5775287085708


In [42]:
#검증 데이터로 예측하기
y_pred = lr.predict(x_test)
print("가중치 확인")
for index, columns in enumerate(x_train.columns):
    print(f"{columns} = {reg.coef_[index]}")


가중치 확인
연령대코드(5세단위) = -0.0021154736576941484
신장(5cm단위) = 0.008382856140952737
체중(5kg단위) = 0.004039744108514473
허리둘레 = 0.02586561239950298
수축기혈압 = -0.003622269066452875
이완기혈압 = -0.004789193384671206
식전혈당(공복혈당) = -0.0018193256235434347
혈색소 = 0.9889344164631596
요단백 = -0.18998982863839814
혈청크레아티닌 = -0.9096387125982947
혈청지오티(AST) = 0.04906444369407475
혈청지피티(ALT) = 0.04162509002268398
감마지티피 = 0.14300268237983213
흡연상태 = -0.006545718172645947
음주여부 = 0.005209544960069898


In [43]:
#주요 feature 삭제 후 rmse 확인하기
rmse_2= mean_squared_error(y_test, y_pred, squared=False)
#가중치 큰 특성 삭제했을 때 영향을 많이 받는지 확인
print(f'''주요 Feature 삭제 후 rmse={round(rmse_2,3)}''')

주요 Feature 삭제 후 rmse=37.185


In [ ]:
# 주요 feature를 삭제했을 때 rmse가 많이 오른 것을 볼 수 있다. 